### Robustness of the Measure (HHI)
- use entropy instead of HHI

### 1) Entropy Calculation Example

In [5]:
import math

def calculate_entropy(probabilities):
    """ Calculate the Shannon entropy of a given list of probabilities. """
    entropy = 0
    for p in probabilities:
        if p > 0:
            entropy += p * math.log(p, 2)
    return -entropy

# Example probabilities
probabilities = [0.3, 0.3, 0.4]

# Calculate entropy
entropy = calculate_entropy(probabilities)
print(f"The entropy of the system is: {entropy}")

The entropy of the system is: 1.5709505944546684


In [1]:
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pandas as pd
from nltk import FreqDist
import pickle
import math
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
with open('/data1/StackOverflow/Tag_Analysis/df_tags_2023.pickle', 'rb') as fr:
    df_tags = pickle.load(fr)
def wc(text):
    """
    Cleaning function to be used with our first wordcloud
    """
    
    if text:
        tags = text.replace('><',' ')
        tags = tags.replace('-','')
        tags = tags.replace('.','DOT')
        tags = tags.replace('c++','Cpp')
        tags = tags.replace('c#','Csharp')
        tags = tags.replace('>','')
        return tags.replace('<','')
    else:
        return 'None'
    
def clean_tags(text):
    """
    Cleaning function for tags
    """
    
    if text:
        tags = text.replace('><',' ')
        tags = tags.replace('>','')
        return tags.replace('<','')
    else:
        return 'None'
    
def tag_freq(data):
    tags = data['tags'].str.replace('[\["\]]', '', regex=True)
    tags = [tag for i in tags.apply(lambda x: wc(x)) for tag in i.split(', ')]
    result = FreqDist(tags)
    return result
# Preprocessing
df_tags['creation_date'] = pd.to_datetime(df_tags['creation_date'])
df_tags['year_month'] = df_tags['creation_date'].dt.to_period('D')
df_tags['year_month'] = df_tags['year_month'].astype(str)
year_month = df_tags.year_month.unique()

In [2]:
# load pickle
with open('/data1/StackOverflow/diff_in_diff/daily_tagShare_modified.pickle', 'rb') as fr:
    all_keys = pickle.load(fr)

In [3]:
all_keys

,index,tag,tag_2020_11_30,tagShare_2020_11_30,tag_2020_12_01,tagShare_2020_12_01,tag_2020_12_02,tagShare_2020_12_02,tag_2020_12_03,tagShare_2020_12_03,...,tag_2023_08_30,tagShare_2023_08_30,tag_2023_08_31,tagShare_2023_08_31,tag_2023_09_01,tagShare_2023_09_01,tag_2023_09_02,tagShare_2023_09_02,tag_2023_09_03,tagShare_2023_09_03
0,amazonwebservices,52132,47.0,0.315352,59.0,0.368819,57.0,0.352505,86.0,0.493657,...,47.0,0.409087,57.0,0.503890,38.0,0.389744,25.0,0.41625,13.0,0.689655
1,forloop,16939,19.0,0.127483,21.0,0.131275,27.0,0.166976,32.0,0.183686,...,11.0,0.095744,4.0,0.035361,4.0,0.041026,3.0,0.04995,3.0,0.159151
2,foreach,3614,9.0,0.060386,5.0,0.031256,3.0,0.018553,6.0,0.034441,...,3.0,0.026112,2.0,0.017680,1.0,0.010256,1.0,0.01665,NaN,NaN
3,terraform,11491,13.0,0.087225,12.0,0.075014,10.0,0.061843,8.0,0.045922,...,13.0,0.113152,17.0,0.150283,13.0,0.133333,2.0,0.03330,NaN,NaN
4,awsglue,2203,3.0,0.020129,5.0,0.031256,2.0,0.012369,1.0,0.005740,...,2.0,0.017408,NaN,NaN,2.0,0.020513,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51948,applespeech,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.053050
51949,jsonloader,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.053050
51950,livescript,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.053050
51951,nxhtml,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.053050


In [6]:
# Measure score
entropy_Score = []
# Calculate HHI for each monthly tag share column.
for i in range(3, all_keys.shape[1], 2):
    entropy_Score.append(calculate_entropy(all_keys.iloc[:, i]))
result_entropy = pd.DataFrame({'year_month':year_month, 'ent_Score':entropy_Score})

In [7]:
result_entropy

,year_month,ent_Score
0,2020-11-30,299.247855
1,2020-12-01,304.216314
2,2020-12-02,304.889077
3,2020-12-03,309.631880
4,2020-12-04,307.007783
...,...,...
1003,2023-08-30,344.541147
1004,2023-08-31,340.270583
1005,2023-09-01,337.331637
1006,2023-09-02,293.754004


In [17]:
entropy = result_entropy[(result_entropy['year_month'] > '2021-08-31') &
           (result_entropy['year_month'] < '2023-09-01')].reset_index(drop = True)

In [18]:
entropy

,year_month,ent_Score
0,2021-09-01,299.305018
1,2021-09-02,296.878176
2,2021-09-03,301.318720
3,2021-09-04,254.122646
4,2021-09-05,254.078037
...,...,...
725,2023-08-27,292.230283
726,2023-08-28,337.791827
727,2023-08-29,341.701967
728,2023-08-30,344.541147


In [20]:
entropy.to_csv("entropy_did.csv", index = False)

### 3) Diff-in-Diff

In [63]:
entropy = list(result_entropy[(result_entropy['year_month'] > '2021-08-31') &
           (result_entropy['year_month'] < '2023-09-01')].reset_index().ent_Score) # fixed datetime
# Split Data
control_data = pd.DataFrame({'HHI' : entropy[:365],
              'T_d': [0]*len(entropy[:365]),
              'P_t' : [0]*90 + [1]*275})
treated_data = pd.DataFrame({'HHI' : entropy[365:],
              'T_d': [1]*len(entropy[365:]),
              'P_t' : [0]*90 + [1]*275})
df_did = pd.concat([control_data, treated_data], axis = 0).reset_index(drop = True)
# Add date and month feature
df_did['date'] = result_entropy[(result_entropy['year_month'] > '2021-08-31') &
           (result_entropy['year_month'] < '2023-09-01')].reset_index().year_month
df_did['month'] = pd.to_datetime(df_did['date']).dt.month
# Apply log
df_did['ln_y'] = np.log(df_did['HHI'])

In [64]:
df_did

,HHI,T_d,P_t,date,month,ln_y
0,299.305018,0,0,2021-09-01,9,5.701463
1,296.878176,0,0,2021-09-02,9,5.693322
2,301.318720,0,0,2021-09-03,9,5.708169
3,254.122646,0,0,2021-09-04,9,5.537817
4,254.078037,0,0,2021-09-05,9,5.537641
...,...,...,...,...,...,...
725,292.230283,1,1,2023-08-27,8,5.677542
726,337.791827,1,1,2023-08-28,8,5.822430
727,341.701967,1,1,2023-08-29,8,5.833939
728,344.541147,1,1,2023-08-30,8,5.842214


### 4) Model Fitting

In [65]:
sm.ols('ln_y ~ T_d + P_t + T_d * P_t + C(month)', df_did).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,5.6379,0.049,115.858,0.000,5.542,5.733
C(month)[T.2],0.0158,0.012,1.293,0.196,-0.008,0.040
C(month)[T.3],0.0163,0.012,1.373,0.170,-0.007,0.040
C(month)[T.4],-0.0047,0.012,-0.391,0.696,-0.028,0.019
C(month)[T.5],-0.0018,0.012,-0.155,0.877,-0.025,0.022
C(month)[T.6],0.0075,0.012,0.625,0.532,-0.016,0.031
C(month)[T.7],-0.0036,0.012,-0.298,0.765,-0.027,0.020
C(month)[T.8],0.0128,0.012,1.075,0.283,-0.011,0.036
C(month)[T.9],0.0208,0.049,0.422,0.673,-0.076,0.117
C(month)[T.10],0.0157,0.049,0.319,0.750,-0.081,0.112


In [66]:
sm.ols('ln_y ~ T_d + P_t + T_d * P_t + C(month)', df_did).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   ln_y   R-squared:                       0.377
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     30.96
Date:                Thu, 18 Apr 2024   Prob (F-statistic):           2.42e-64
Time:                        11:48:38   Log-Likelihood:                 952.80
No. Observations:                 730   AIC:                            -1876.
Df Residuals:                     715   BIC:                            -1807.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          5.6379      0.049    115.858      0.000       5.542       5.733
C(month)[T.2]      0.0158      0.012      1.293      0.196      -0.008       0.040
C(month)[T.3]      0.0163      0.012      1.373      0.170      -0.007       0.040
C(month)[T.4]     -0.0047      0.012     -0.391      0.696      -0.028       0.019
C(month)[T.5]     -0.0018      0.012     -0.155      0.877      -0.025       0.022
C(month)[T.6]      0.0075      0.012      0.625      0.532      -0.016       0.031
C(month)[T.7]     -0.0036      0.012     -0.298      0.765      -0.027       0.020
C(month)[T.8]      0.0128      0.012      1.075      0.283      -0.011       0.036
C(month)[T.9]      0.0208      0.049      0.422      0.673      -0.076       0.117
C(month)[T.10]     0.0157      0.049      0.319      0.750      -0.081       0.112
C(month)[T.11]     0.0390      0.048      0.819      0.413      -0.054       0.133
C(month)[T.12]    -0.0191      0.012     -1.607      0.109      -0.043       0.004
T_d                0.0657      0.010      6.644      0.000       0.046       0.085
P_t                0.0257      0.048      0.534      0.593      -0.069       0.120
T_d:P_t            0.0405      0.011      3.555      0.000       0.018       0.063
==============================================================================
Omnibus:                      139.223   Durbin-Watson:                   1.224
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              116.006
Skew:                          -0.885   Prob(JB):                     6.45e-26
Kurtosis:                       2.174   Cond. No.                         65.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""